In [2]:
using DataFrames, Plots, LowRankModels, CSV, LinearAlgebra, Statistics, ScikitLearn, Random, Distributions
include("proxgrad.jl")

proxgrad_const

In [3]:
df = CSV.read("FinalDataSet.csv")

,Column1,Date,Reddit Post Count,Twitter Post Count,Company,Reddit Upvote Sum
,Int64,Date…,Int64,Int64,String,Int64
1,0,0020-07-01,1,317034,FB,0
2,1,0020-07-02,2,297197,FB,12
3,2,0020-07-06,0,278204,FB,0
4,3,0020-07-07,1,309570,FB,2
5,4,0020-07-08,3,338002,FB,81
6,5,0020-07-09,0,285488,FB,0
7,6,0020-07-10,0,329350,FB,0
8,7,0020-07-13,1,321903,FB,1
9,8,0020-07-14,1,314377,FB,3


## Fit on all data at once. Logistic Loss, Quadratic Regularizer

In [6]:
data = df[shuffle(1:end), :]

train_proportion = 0.6
n = size(data, 1)

ntrain = convert(Int, round(train_proportion*n))

target = data[:Returns]
features = data[:, [:"Reddit Post Count", :"Twitter Post Count", :"Reddit Upvote Sum"]]

train_x = features[1:ntrain, :]
test_x = features[ntrain+1:end, :]
train_y = target[1:ntrain]
test_y = target[ntrain+1:end]

ntrain

192

In [20]:
X_train = [convert(Matrix, train_x) ones(ntrain)]
X_test = [convert(Matrix, test_x) ones(size(test_x, 1))]

ybool_train = (train_y .>= 0) 
ybool_test = (test_y .>= 0)

n = size(ybool_test, 1)

loss = 1/ntrain*LogisticLoss()
λ = 0.1
reg = λ*QuadReg()
w = proxgrad(loss, reg, X_train, ybool_train, maxiters=200000, stepsize = 0.01)

yhat = impute(loss, X_test*w)

(n - sum(yhat .== ybool_test)) / n

0.4765625

## Separate model for each company. Logistic Loss, Quadratic Regularizer

In [25]:
FB = df[(df[:Company] .== "FB"),:]
AAPL = df[(df[:Company] .== "AAPL"),:]
AMZN = df[(df[:Company] .== "AMZN"),:]
NFLX = df[(df[:Company] .== "NFLX"),:]
GOOG = df[(df[:Company] .== "GOOG"),:]

,Column1,Date,Reddit Post Count,Twitter Post Count,Company,Reddit Upvote Sum
,Int64,Date…,Int64,Int64,String,Int64
1,256,0020-07-01,0,1286,GOOG,0
2,257,0020-07-02,4,1070,GOOG,38
3,258,0020-07-06,1,1073,GOOG,8
4,259,0020-07-07,1,1187,GOOG,3
5,260,0020-07-08,1,1094,GOOG,1
6,261,0020-07-09,2,1081,GOOG,4
7,262,0020-07-10,2,1425,GOOG,2
8,263,0020-07-13,0,1326,GOOG,0
9,264,0020-07-14,3,1268,GOOG,26


### Facebook

In [27]:
fb_data = FB[shuffle(1:end), :]

train_proportion = 0.6
n = size(fb_data, 1)

ntrain = convert(Int, round(train_proportion*n))

target = fb_data[:Returns]
features = fb_data[:, [:"Reddit Post Count", :"Twitter Post Count", :"Reddit Upvote Sum"]]

train_x = features[1:ntrain, :]
test_x = features[ntrain+1:end, :]
train_y = target[1:ntrain]
test_y = target[ntrain+1:end]

ntrain

38

In [28]:
X_train = [convert(Matrix, train_x) ones(ntrain)]
X_test = [convert(Matrix, test_x) ones(size(test_x, 1))]

ybool_train = (train_y .>= 0) 
ybool_test = (test_y .>= 0)

n = size(ybool_test, 1)

loss = 1/ntrain*LogisticLoss()
λ = 0.1
reg = λ*QuadReg()
w = proxgrad(loss, reg, X_train, ybool_train, maxiters=200000, stepsize = 0.01)

yhat = impute(loss, X_test*w)

(n - sum(yhat .== ybool_test)) / n

0.5

### Apple

In [33]:
aapl_data = AAPL[shuffle(1:end), :]

train_proportion = 0.6
n = size(aapl_data, 1)

ntrain = convert(Int, round(train_proportion*n))

target = aapl_data[:Returns]
features = aapl_data[:, [:"Reddit Post Count", :"Twitter Post Count", :"Reddit Upvote Sum"]]

train_x = features[1:ntrain, :]
test_x = features[ntrain+1:end, :]
train_y = target[1:ntrain]
test_y = target[ntrain+1:end]

ntrain

38

In [34]:
X_train = [convert(Matrix, train_x) ones(ntrain)]
X_test = [convert(Matrix, test_x) ones(size(test_x, 1))]

ybool_train = (train_y .>= 0) 
ybool_test = (test_y .>= 0)

n = size(ybool_test, 1)

loss = 1/ntrain*LogisticLoss()
λ = 0.1
reg = λ*QuadReg()
w = proxgrad(loss, reg, X_train, ybool_train, maxiters=200000, stepsize = 0.01)

yhat = impute(loss, X_test*w)

(n - sum(yhat .== ybool_test)) / n

0.38461538461538464

### Amazon

In [39]:
amzn_data = AMZN[shuffle(1:end), :]

train_proportion = 0.6
n = size(amzn_data, 1)

ntrain = convert(Int, round(train_proportion*n))

target = amzn_data[:Returns]
features = amzn_data[:, [:"Reddit Post Count", :"Twitter Post Count", :"Reddit Upvote Sum"]]

train_x = features[1:ntrain, :]
test_x = features[ntrain+1:end, :]
train_y = target[1:ntrain]
test_y = target[ntrain+1:end]

ntrain

38

In [40]:
X_train = [convert(Matrix, train_x) ones(ntrain)]
X_test = [convert(Matrix, test_x) ones(size(test_x, 1))]

ybool_train = (train_y .>= 0) 
ybool_test = (test_y .>= 0)

n = size(ybool_test, 1)

loss = 1/ntrain*LogisticLoss()
λ = 0.1
reg = λ*QuadReg()
w = proxgrad(loss, reg, X_train, ybool_train, maxiters=200000, stepsize = 0.01)

yhat = impute(loss, X_test*w)

(n - sum(yhat .== ybool_test)) / n

0.46153846153846156

### Netflix

In [43]:
nflx_data = NFLX[shuffle(1:end), :]

train_proportion = 0.6
n = size(nflx_data, 1)

ntrain = convert(Int, round(train_proportion*n))

target = nflx_data[:Returns]
features = nflx_data[:, [:"Reddit Post Count", :"Twitter Post Count", :"Reddit Upvote Sum"]]

train_x = features[1:ntrain, :]
test_x = features[ntrain+1:end, :]
train_y = target[1:ntrain]
test_y = target[ntrain+1:end]

ntrain

38

In [44]:
X_train = [convert(Matrix, train_x) ones(ntrain)]
X_test = [convert(Matrix, test_x) ones(size(test_x, 1))]

ybool_train = (train_y .>= 0) 
ybool_test = (test_y .>= 0)

n = size(ybool_test, 1)

loss = 1/ntrain*LogisticLoss()
λ = 0.1
reg = λ*QuadReg()
w = proxgrad(loss, reg, X_train, ybool_train, maxiters=200000, stepsize = 0.01)

yhat = impute(loss, X_test*w)

(n - sum(yhat .== ybool_test)) / n

0.5384615384615384

### Google

In [50]:
goog_data = GOOG[shuffle(1:end), :]

train_proportion = 0.6
n = size(goog_data, 1)

ntrain = convert(Int, round(train_proportion*n))

target = goog_data[:Returns]
features = goog_data[:, [:"Reddit Post Count", :"Twitter Post Count", :"Reddit Upvote Sum"]]

train_x = features[1:ntrain, :]
test_x = features[ntrain+1:end, :]
train_y = target[1:ntrain]
test_y = target[ntrain+1:end]

ntrain

38

In [51]:
X_train = [convert(Matrix, train_x) ones(ntrain)]
X_test = [convert(Matrix, test_x) ones(size(test_x, 1))]

ybool_train = (train_y .>= 0) 
ybool_test = (test_y .>= 0)

n = size(ybool_test, 1)

loss = 1/ntrain*LogisticLoss()
λ = 0.1
reg = λ*QuadReg()
w = proxgrad(loss, reg, X_train, ybool_train, maxiters=200000, stepsize = 0.01)

yhat = impute(loss, X_test*w)

(n - sum(yhat .== ybool_test)) / n

0.5769230769230769